In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandexo.engine.justdoit as jdi # THIS IS THE HOLY GRAIL OF PANDEXO
import numpy as np
import os

## Editting Inputs

### Step 1) Load in a blank exoplanet dictionary

To start, load in a blank exoplanet dictionary with empty keys. You will fill these out for yourself in the next step. 

In [2]:
exo_dict = jdi.load_exo_dict()
jdi.load_exo_dict()

Replace all inputs before feeding to run_modes:
Replace all inputs before feeding to run_modes:


{'observation': {'fraction': 'time in/out',
  'noccultations': 'num transits',
  'noise_floor': 'constant number or file name',
  'sat_level': 'in % sat level',
  'transit_duration': 'in seconds',
  'wave_bin': 'in micron'},
 'planet': {'exopath': 'file path',
  'f_unit': 'rp/r* or fp/f*',
  'type': 'user',
  'w_unit': 'Angs,cm,um,cm or Hz'},
 'star': {'f_unit': 'W/m2/um, FLAM, Jy, or erg/s/cm2/Hz',
  'logg': 'cgs',
  'mag': 'magnitude',
  'metal': 'in log Fe/H',
  'ref_wave': 'corresponding ref wave',
  'starpath': 'file path',
  'temp': 'Only if phoenix, (in K)',
  'type': 'user or phoenix',
  'w_unit': 'Angs,cm,um,cm or Hz'}}

#### Edit exoplanet observation inputs
Editting each keys are annoying. But, do this carefully or it could result in nonsense runs

In [3]:
exo_dict['calculation'] = '2d extract'
exo_dict['observation']['sat_level'] = 80 #saturation level in percent of full well 
exo_dict['observation']['transit_duration'] = 2.0*60.0*60.0 #transit duration in seconds
exo_dict['observation']['noccultations'] = 2 #number of transits 
#fixed binning. I usually suggest ZERO binning.. you can always bin later 
#without having to redo the calcualtion
exo_dict['observation']['wave_bin'] = 0.0 
exo_dict['observation']['fraction'] = 1.0 #fraction of time in transit versus out = in/out
exo_dict['observation']['noise_floor'] = 0 #this can be a fixed level or it can be a filepath 
#to a wavelength dependent noise floor solution

#### Edit exoplanet star inputs
Note... If you select 'phoenix' you **do not** have to provide a starpath, w_unit or f_unit, but you **do** have to provide a temp, metal and logg. If you select 'user' you **do not** need to provide a temp, metal and logg, but you **do** need to provide units and starpath. 

In [4]:
exo_dict['star']['type'] = 'phoenix' #phoenix or user (if you have your own)
exo_dict['star']['mag'] = 8.0 #magnitude of the system
exo_dict['star']['ref_wave'] = 1.25 #always add this in micron
exo_dict['star']['temp'] = 5500 #in K 
exo_dict['star']['metal'] = 0.0 # as log Fe/H
exo_dict['star']['logg'] = 4.0 #log surface gravity cgs

#### Edit exoplanet planet inputs

In [5]:
exo_dict['planet']['type'] = 'user' #only option.. until a dataase is made
exo_dict['planet']['exopath'] = '/Users/nbatalh1/Desktop/Simulations/wasp12b.txt'
exo_dict['planet']['w_unit'] = 'cm' 
exo_dict['planet']['f_unit'] = 'rp/r*' #this is actually (rp/r*)^2

### Step 2) Load in instrument dictionary (optional)
 
Step 2 is optional because PandExo has the functionality to automatically load in instrument dictionaries. Skip this if you plan on observing with one of the following: 
- NIRCam F444W
- NIRSpec Prism
- NIRSpec G395M
- NIRSpec G395H
- NIRSpec G235H
- NIRSpec G235M
- NIRCam F322W
- NIRSpec G140M
- NIRSpec G140H
- MIRI LRS
- NIRISS SOSS_Or1
- NIRISS SOSS_Or2

In [6]:
jdi.print_instruments()

Choose from the following:
['NIRCam F444W', 'NIRSpec Prism', 'NIRSpec G395M', 'NIRSpec G395H', 'NIRSpec G235H', 'NIRSpec G235M', 'NIRCam F322W', 'NIRSpec G140M', 'NIRSpec G140H', 'MIRI LRS', 'NIRISS SOSS_Or1', 'NIRISS SOSS_Or2']


In [7]:
inst_dict = jdi.load_mode_dict('NIRCam F444W')

## Running PandExo

You have **four options** for running PandExo. All of them are accessed through attribute **jdi.run_pandexo**. See examples below. 

`    jdi.run_pandexo(exo, inst, param_space = 0, param_range = 0,save_file = True,
                            output_path=os.getcwd(), output_file = '')`

See wiki Attributes for more thorough explanation fo inputs


### Option 1- Run single instrument mode, single planet
If you forget which instruments are available run **jdi.print_isntruments()** and pick one 

In [8]:
jdi.print_instruments()

Choose from the following:
['NIRCam F444W', 'NIRSpec Prism', 'NIRSpec G395M', 'NIRSpec G395H', 'NIRSpec G235H', 'NIRSpec G235M', 'NIRCam F322W', 'NIRSpec G140M', 'NIRSpec G140H', 'MIRI LRS', 'NIRISS SOSS_Or1', 'NIRISS SOSS_Or2']


In [9]:
result = jdi.run_pandexo(exo_dict,['NIRCam F444W'])

Running Single Case for: NIRCam F444W


TypeError: loopingL() takes exactly 4 arguments (5 given)

### Option 2- Run single instrument mode (with user dict), single planet
This is the same thing as option 1 but instead of feeding it a list of keys, you can feed it a instrument dictionary (this is for users who wanted to simulate something NOT pre defined within pandexo)

In [ ]:
result = jdi.run_pandexo(exo_dict, inst_dict)

### Option 3- Run several modes, single planet
Use several modes from **print_isntruments()** options. 

In [ ]:
#choose select 
result = jdi.run_pandexo(exo_dict,['NIRCam F444W','NIRCam F322W2','MIRI LRS'],
               output_path = '/Users/nbatalh1/Desktop/JWSTFUN')
#run all 
result = jdi.run_pandexo(exo_dict, ['RUN ALL'], save_file = False)

### Option 4- Run single mode, several planet cases
Use a single modes from **print_isntruments()** options. But explore parameter space with respect to **any** parameter in the exo dict. The example below shows how to loop over several planet models

You can loop through anything in the exoplanet dictionary. It will be planet, star or observation followed by whatever you want to loop through in that set. 

i.e. planet+exopath, star+temp, star+metal, star+logg, observation+sat_level.. etc

In [ ]:
#looping over different exoplanet models 
jdi.run_pandexo(exo_dict, ['NIRCam F444W'], param_space = 'planet+exopath',
                param_range = os.listdir('/Users/nbatalha1/all_my_models_here'),
               output_path = '/Users/nbatalh1/Desktop/JWSTFUN')

#looping over different stellar temperatures 
jdi.run_pandexo(exo_dict, ['NIRCam F444W'], param_space = 'star+temp',
                param_range = np.linspace(5000,8000,2),
               output_path = '/Users/nbatalh1/Desktop/JWSTFUN')

#looping over different saturation levels
jdi.run_pandexo(exo_dict, ['NIRCam F444W'], param_space = 'observation+sat_level',
                param_range = np.linspace(.5,1,5),
               output_path = '/Users/nbatalh1/Desktop/JWSTFUN')
